## Importing Libraries

In [1]:
import sys
print(sys.executable)
print(sys.version_info)

from transformers import pipeline
from transformers import RagTokenizer, RagRetriever, RagTokenForGeneration

## Code Segment

Tokenization: First, it initializes a tokenizer using RagTokenizer.from_pretrained("facebook/rag-token-nq"). The RagTokenizer is used to preprocess input text so that it can be fed into the model.

In [2]:
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")

/Users/shotomorisaki/miniconda3/lib/python3.11/site-packages/transformers/models/bart/configuration_bart.py:179: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called f

- Retriever:
1. it initializes a retriever using RagRetriever.from_pretrained("facebook/rag-token-nq", index_name="exact", use_dummy_dataset=True). The RagRetriever is responsible for retrieving relevant passages of text from a knowledge base given an input query. In this case, it's set up to use an exact search index and a dummy dataset.

In [3]:
retriever = RagRetriever.from_pretrained("facebook/rag-token-nq", index_name="exact", use_dummy_dataset=True)
# du -sh ~/.cache/huggingface

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

- Model Initialization: 
1. Then, it initializes the RAG model using RagTokenForGeneration.from_pretrained("facebook/rag-token-nq", retriever=retriever). This model is capable of generating text conditioned on both the input query and retrieved passages.There's a commented-out line that initializes a TensorFlow-compatible version of the model (TFRagTokenForGeneration). Depending on whether you're using PyTorch or TensorFlow, you can choose the appropriate model type.

In [8]:
from transformers import TFRagTokenForGeneration
from transformers import BertConfig, BertModel, BertLMHeadModel

model_facebook = RagTokenForGeneration.from_pretrained("facebook/rag-token-nq", retriever=retriever)
model_bert = BertLMHeadModel.from_pretrained('bert-base-uncased')


/Users/shotomorisaki/miniconda3/lib/python3.11/site-packages/transformers/models/bart/configuration_bart.py:179: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
Some weights of the model checkpoint at facebook/rag-token-nq were not used when initializing RagTokenForGeneration: ['rag.question_encoder.question_encoder.bert_model.pooler.dense.weight', 'rag.question_encoder.question_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing RagTokenForGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RagTokenForGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenc

TypeError: BertLMHeadModel.__init__() got an unexpected keyword argument 'retriever'

- Prepare Input: 
1. The code prepares the input for generation using the tokenizer's prepare_seq2seq_batch method. This method is deprecated, as indicated by the warning, but it's still functional for now. It converts the input query into a format suitable for model input.

In [7]:
input = "who holds the record in 100m freestyle"
input_dict = tokenizer.prepare_seq2seq_batch(input, return_tensors="pt") 
# input_dict = tokenizer(input, return_tensors="pt")


/Users/shotomorisaki/miniconda3/lib/python3.11/site-packages/transformers/models/rag/tokenization_rag.py:87: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  warnings.warn(


- Generate Text:
1. The model generates text based on the prepared input using the generate method. This method generates text based on the input and any context retrieved by the retriever. The generated text is then decoded using the tokenizer's batch_decode method to convert the generated token IDs back into human-readable text.

In [6]:
generated_facebook = model_facebook.generate(input_ids=input_dict["input_ids"]) 
generated_bert = model_bert.generate(input_ids=input_dict["input_ids"]) 
print("-" * 20)
# print("Generated: " + str(generated))
print(tokenizer.batch_decode(generated_facebook, skip_special_tokens=True)[0])
print(tokenizer.batch_decode(generated_bert, skip_special_tokens=True)[0])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/shotomorisaki/miniconda3/lib/python3.11/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(
/Users/shotomorisaki/miniconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


--------------------
 michael phelps
 like cultureua asking boosthaini hundreds Rileya TV TV TV TV TV TV TV TV TV TV
